In [ ]:
# default_exp face_class_ver

In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
#export
# imports
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
from PIL import Image
from torch.nn.functional import cosine_similarity, adaptive_avg_pool2d
from torch.optim import Adam, SGD
from torch.optim.lr_scheduler import StepLR
from sklearn.metrics import roc_auc_score
import pandas as pd
from functools import partial
from hyperopt import fmin, tpe, hp, Trials, STATUS_OK
from hyperopt.pyll.base import scope
import json
from datetime import datetime
import os
import time

from datasets import FaceClassificationDataset, FaceVerificationDataset
from losses import CenterLoss
from models.mobilenet import *
from models.resnet import *
from hyperoptimization import fit_predict
# from axa_hw2p2.datasets import FaceClassificationDataset, FaceVerificationDataset
# from axa_hw2p2.losses import CenterLoss
# from axa_hw2p2.models.mobilenet import *
# from axa_hw2p2.models.resnet import *
# from axa_hw2p2.hyperoptimization import fit_predict

In [ ]:
#export
def main(model, batch_size, sample_size=None):

    # Hyperparameters space
    space = {'model': hp.choice(label='model', options=[model]),
             'in_channels': hp.choice(label='in_channels', options=[3]),
            #  'n_classes': hp.choice(label='n_classes', options=[4000]),
             'batch_size': scope.int(hp.choice(label='batch_size', options=[batch_size])),
             'lr': hp.loguniform(label='lr', low=np.log(5e-2), high=np.log(2e-1)),
             'lr_decay': hp.choice(label='lr_decay', options=[0.98,0.99,1]),
             'n_lr_decay_steps': hp.choice(label='n_lr_decay_steps', options=[1,2,4]),
             'center_loss': hp.choice(label='center_loss', options=[True]),
             'lr_cl': hp.choice(label='lr_cl', options=[0.5]),
             'alpha_cl': hp.choice(label='alpha_cl', options=[0.01,0.1,1]),
             'n_epochs': hp.choice(label='n_epochs', options=[25]),
             'eval_steps': scope.int(hp.choice(label='eval_steps', options=[4])),}

    if sample_size==None:
        space['n_classes'] = hp.choice(label='n_classes', options=[4000])
    else:
        space['n_classes'] = hp.choice(label='n_classes', options=[sample_size])

    # Hyperparameters search
    trials = Trials()
    fmin_objective = partial(fit_predict, verbose=True, trials=trials, sample_size=sample_size)
    fmin(fmin_objective, space=space, 
         algo=tpe.suggest, max_evals=20, trials=trials)

In [ ]:
os.getcwd()
# os.chdir('..')

'/home/alejandroxag/myfiles/deeplearning/hw2p2'

In [ ]:
#export
if __name__ == "__main__": 
    main(model='resnet18', batch_size=512, sample_size=None)


Current directory: /home/alejandroxag/myfiles/deeplearning/hw2p2

15-03-21_17-35-00
alpha_cl                   1
batch_size                 1
center_loss             True
eval_steps                 4
in_channels                3
lr                  0.167076
lr_cl                    0.5
lr_decay                   1
model               resnet18
n_classes               4000
n_epochs                  25
n_lr_decay_steps           2
dtype: object

train_dataset_len: 380638, val_c_dataset_len: 8000, val_v_dataset_len: 8805
==============================Start Fitting==============================
  0%|          | 0/20 [00:10<?, ?trial/s, best loss=?]


KeyboardInterrupt: 